# BPE Algorithm
- BPE은 연속적으로 가장 많이 등장한 글자의 쌍을 찾아서 하나의 글자로 병합하는 방식을 수행하는 압축 알고리즘이다.
- `aaabdaaabac`를 `Z=aa`를 통해 `ZabdZabac`로 병합하는 식이다.

## collections.defaultdict()

In [2]:
# collections.defaultdict()

import collections
 
normalDict = collections.defaultdict(int)  # 값(value)을 자동으로 0으로 채워 준다.
normalKey =["A","B","C"]
 
for item in normalKey:
    normalDict[item]
 
print(normalDict)

defaultdict(<class 'int'>, {'A': 0, 'B': 0, 'C': 0})


In [7]:
# collections.defaultdict()

normalDict = collections.defaultdict(int)
 
print(normalDict)  # 빈 defaultdict

normalDict["a"]  # key로 'a' 추가
print(normalDict)  # 자동으로 값 0

normalDict["v"]  # key로 'v' 추가
print(normalDict)  # 자동으로 값 0

defaultdict(<class 'int'>, {})
defaultdict(<class 'int'>, {'a': 0})
defaultdict(<class 'int'>, {'a': 0, 'v': 0})


- `collections.defaultdict()`는 딕셔너리(dictionary)와 유사하지만 key에 대한 값이 없을 경우 미리 지정해 놓은 초기(default) 값을 반환하는 dictionary다.
- `defaultdict()`의 인자 값으로 `lamda:0`을 넣어도 되지만, 위와 같이 `int`를 넣어도 기본 값으로 0이 입력된다.

## BPE Training & Applying

In [ ]:
import re, collections

- collections 라이브러리 : 데이터 개수 셀 때 유용한 라이브러리

In [8]:
num_merges = 10  # BPE 수행 횟수

In [9]:
# 딕셔너리의 단어를 글자(character) 단위로 분리 : 빈도수

dictionary = {'l o w </w>' : 5,
              'l o w e r </w>' : 2,
              'n e w e s t </w>' : 6,
              'w i d e s t </w>' : 3
              }

- **N-gram** : n개의 단어 집합으로 묶어 주는 것
 - **Unigram** : 우리들은 / 밥을 /먹었고, / 나는 / 공부중이다.
 - **Bigram** : 우리들은 밥을 / 먹었고, 나는 / 공부 중이다. \
 - **Trigram** : 우리들은 밥을 먹었고, / 나는 공부 중이다.

In [ ]:
# unigram의 pair들의 빈도수를 카운트

def get_stats(dictionary):
  pairs = collections.defaultdict(int)  # 값(value)을 0으로 자동 할당

  for word, freq in dictionary.items():
    symbols = word.split()

    # print(symbols)
    # ['l', 'o', 'w', '</w>']
    # ['l', 'o', 'w', 'e', 'r', '</w>']
    # ['n', 'e', 'w', 'e', 's', 't', '</w>']
    # ['w', 'i', 'd', 'e', 's', 't', '</w>']

    for i in range(len(symbols)-1):  # '</w>'는 빼야 하므로 -1

      # print(range(len(symbols)-1))
      # range(0, 3)  # low 3글자
      # range(0, 5)
      # range(0, 6)
      # range(0, 6)

      pairs[symbols[i], symbols[i+1]] += freq

  print('현재 pair들의 빈도 수 :', dict(pairs))
  return pairs

In [ ]:
# 최빈도 pair를 merge하는 함수

def merge_dictionary(pair, v_in):
  v_out = {}
  bigram = re.escape(' '.join(pair))
  p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
  for word in v_in:
    w_out = p.sub(''.join(pair), word)
    v_out[w_out] = v_in[word]
  return v_out

In [ ]:
bpe_codes = {}
bpe_codes_reverse = {}

In [ ]:
# 최빈도 pair의 merge를 num_merges(= 10)만큼 반복

for i in range(num_merges):
  print(">> Step {0}".format(i+1))
  pairs = get_stats(dictionary)
  best = max(pairs, key = pairs.get)
  dictionary = merge_dictionary(best, dictionary)

  bpe_codes[best] = i
  bpe_codes_reverse[best[0] + best[1]] = best

  print("new merge : {}".format(best))
  print("dictionary : {}".format(dictionary))

>> Step 1
현재 pair들의 빈도 수 : {('l', 'o'): 7, ('o', 'w'): 7, ('w', '</w>'): 5, ('w', 'e'): 8, ('e', 'r'): 2, ('r', '</w>'): 2, ('n', 'e'): 6, ('e', 'w'): 6, ('e', 's'): 9, ('s', 't'): 9, ('t', '</w>'): 9, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'e'): 3}
new merge : ('e', 's')
dictionary : {'l o w </w>': 5, 'l o w e r </w>': 2, 'n e w es t </w>': 6, 'w i d es t </w>': 3}
>> Step 2
현재 pair들의 빈도 수 : {('l', 'o'): 7, ('o', 'w'): 7, ('w', '</w>'): 5, ('w', 'e'): 2, ('e', 'r'): 2, ('r', '</w>'): 2, ('n', 'e'): 6, ('e', 'w'): 6, ('w', 'es'): 6, ('es', 't'): 9, ('t', '</w>'): 9, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'es'): 3}
new merge : ('es', 't')
dictionary : {'l o w </w>': 5, 'l o w e r </w>': 2, 'n e w est </w>': 6, 'w i d est </w>': 3}
>> Step 3
현재 pair들의 빈도 수 : {('l', 'o'): 7, ('o', 'w'): 7, ('w', '</w>'): 5, ('w', 'e'): 2, ('e', 'r'): 2, ('r', '</w>'): 2, ('n', 'e'): 6, ('e', 'w'): 6, ('w', 'est'): 6, ('est', '</w>'): 9, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'est'): 3}
new merge : ('est', '</w>')


In [ ]:
print(bpe_codes)  # bpe_code를 출력하면 merge했던 기록이 출력된다.

{('e', 's'): 0, ('es', 't'): 1, ('est', '</w>'): 2, ('l', 'o'): 3, ('lo', 'w'): 4, ('n', 'e'): 5, ('ne', 'w'): 6, ('new', 'est</w>'): 7, ('low', '</w>'): 8, ('w', 'i'): 9}


## BPE의 OOV 대처

In [ ]:
def get_pairs(word):
  # 단어의 기호 쌍 집합 반환
  # 단어는 기호(variable-length strings)의 tuple로 표현된다.
  
  pairs = set()
  prev_char = word[0]
  for char in word[1:]:
    pairs.add((prev_char, char))
    prev_char = char
  return pairs

In [ ]:
def encode(orig):
  # 연속으로 적용되는 BPE merge 수행 목록을 기반으로 단어를 인코딩

  word = tuple(orig) + ('</w>',)
  # display(Markdown("__word split into characters:__ <tt>{}</tt>".format(word)))
  print("__word split into characters:__ <tt>{}</tt>".format(word))

  pairs = get_pairs(word)    

  if not pairs:
      return orig

  iteration = 0
  while True:
      iteration += 1
      # display(Markdown("__Iteration {}:__".format(iteration)))
      print("__Iteration {}:__".format(iteration))

      print("bigrams in the word: {}".format(pairs))
      bigram = min(pairs, key = lambda pair: bpe_codes.get(pair, float('inf')))
      print("candidate for merging: {}".format(bigram))
      if bigram not in bpe_codes:
          # display(Markdown("__Candidate not in BPE merges, algorithm stops.__"))
          print("__Candidate not in BPE merges, algorithm stops.__")
          break
      first, second = bigram
      new_word = []
      i = 0
      while i < len(word):
        try:
          j = word.index(first, i)
          new_word.extend(word[i:j])
          i = j
        except:
          new_word.extend(word[i:])
          break

        if word[i] == first and i < len(word)-1 and word[i+1] == second:
          new_word.append(first+second)
          i += 2
        else:
          new_word.append(word[i])
          i += 1
      new_word = tuple(new_word)
      word = new_word
      print("word after merging: {}".format(word))
      if len(word) == 1:
          break
      else:
          pairs = get_pairs(word)

  # 특별 토큰인 </w>는 출력하지 않는다.
  if word[-1] == '</w>':
      word = word[:-1]
  elif word[-1].endswith('</w>'):
      word = word[:-1] + (word[-1].replace('</w>',''),)

  return word

In [ ]:
encode("loki")

__word split into characters:__ <tt>('l', 'o', 'k', 'i', '</w>')</tt>
__Iteration 1:__
bigrams in the word: {('i', '</w>'), ('l', 'o'), ('k', 'i'), ('o', 'k')}
candidate for merging: ('l', 'o')
word after merging: ('lo', 'k', 'i', '</w>')
__Iteration 2:__
bigrams in the word: {('i', '</w>'), ('k', 'i'), ('lo', 'k')}
candidate for merging: ('i', '</w>')
__Candidate not in BPE merges, algorithm stops.__


('lo', 'k', 'i')

- 현재 서브워드 단어 집합에는 'lo'가 존재하므로, 'lo'는 유지하고 'k'와 'i'는 분리시킨다.

In [ ]:
encode("lowest")

__word split into characters:__ <tt>('l', 'o', 'w', 'e', 's', 't', '</w>')</tt>
__Iteration 1:__
bigrams in the word: {('e', 's'), ('t', '</w>'), ('s', 't'), ('w', 'e'), ('o', 'w'), ('l', 'o')}
candidate for merging: ('e', 's')
word after merging: ('l', 'o', 'w', 'es', 't', '</w>')
__Iteration 2:__
bigrams in the word: {('w', 'es'), ('t', '</w>'), ('o', 'w'), ('es', 't'), ('l', 'o')}
candidate for merging: ('es', 't')
word after merging: ('l', 'o', 'w', 'est', '</w>')
__Iteration 3:__
bigrams in the word: {('l', 'o'), ('w', 'est'), ('est', '</w>'), ('o', 'w')}
candidate for merging: ('est', '</w>')
word after merging: ('l', 'o', 'w', 'est</w>')
__Iteration 4:__
bigrams in the word: {('w', 'est</w>'), ('l', 'o'), ('o', 'w')}
candidate for merging: ('l', 'o')
word after merging: ('lo', 'w', 'est</w>')
__Iteration 5:__
bigrams in the word: {('w', 'est</w>'), ('lo', 'w')}
candidate for merging: ('lo', 'w')
word after merging: ('low', 'est</w>')
__Iteration 6:__
bigrams in the word: {('low'

('low', 'est')

- 현재 서브워드 단어 집합에 'low'와 'est'가 존재하므로, 'low'와 'est'를 분리시킨다.

In [ ]:
encode("lowing")

__word split into characters:__ <tt>('l', 'o', 'w', 'i', 'n', 'g', '</w>')</tt>
__Iteration 1:__
bigrams in the word: {('o', 'w'), ('w', 'i'), ('n', 'g'), ('g', '</w>'), ('l', 'o'), ('i', 'n')}
candidate for merging: ('l', 'o')
word after merging: ('lo', 'w', 'i', 'n', 'g', '</w>')
__Iteration 2:__
bigrams in the word: {('lo', 'w'), ('w', 'i'), ('g', '</w>'), ('n', 'g'), ('i', 'n')}
candidate for merging: ('lo', 'w')
word after merging: ('low', 'i', 'n', 'g', '</w>')
__Iteration 3:__
bigrams in the word: {('n', 'g'), ('i', 'n'), ('g', '</w>'), ('low', 'i')}
candidate for merging: ('n', 'g')
__Candidate not in BPE merges, algorithm stops.__


('low', 'i', 'n', 'g')

- 현재 서브워드 단어 집합에 'low'가 존재하지만, 'i', 'n', 'g'의 바이그램 조합으로 이루어진 서브워드는 존재하지 않으므로 'i', 'n', 'g'로 전부 분리한다.

In [ ]:
encode("highing")

__word split into characters:__ <tt>('h', 'i', 'g', 'h', 'i', 'n', 'g', '</w>')</tt>
__Iteration 1:__
bigrams in the word: {('h', 'i'), ('g', '</w>'), ('i', 'g'), ('n', 'g'), ('i', 'n'), ('g', 'h')}
candidate for merging: ('h', 'i')
__Candidate not in BPE merges, algorithm stops.__


('h', 'i', 'g', 'h', 'i', 'n', 'g')

- 훈련 데이터 중에 'highing'은 서브워드가 전혀 존재하지 않는다. 따라서 모든 글자(character)가 분리된다.